In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Stats Information

In [7]:
file_path = './../uploads/ADANIPORTS_day_data.csv'
data_df = pd.read_csv(file_path)
data_df = data_df.drop(data_df.columns[0], axis=1)
data_df['date'] = pd.to_datetime(data_df['date'])
data_df.head()

,date,close,high,low,open,volume
0,2015-01-01 00:00:00+05:30,319.55,322.50,316.25,319.00,1456204
1,2015-01-02 00:00:00+05:30,319.35,325.80,318.05,319.35,2894058
2,2015-01-05 00:00:00+05:30,323.80,327.50,319.35,320.45,2099786
3,2015-01-06 00:00:00+05:30,321.85,331.45,315.60,321.64,3672197
4,2015-01-07 00:00:00+05:30,321.10,328.70,317.39,321.95,2981544


In [5]:
data_df['year'],data_df['month'],data_df['day'],data_df['week'] =  data_df['date'].dt.year, data_df['date'].dt.month, data_df['date'].dt.day, data_df['date'].dt.isocalendar().week
data_df.head()

,date,close,high,low,open,volume,year,month,day,week
0,2015-01-01 00:00:00+05:30,319.55,322.50,316.25,319.00,1456204,2015,1,1,1
1,2015-01-02 00:00:00+05:30,319.35,325.80,318.05,319.35,2894058,2015,1,2,1
2,2015-01-05 00:00:00+05:30,323.80,327.50,319.35,320.45,2099786,2015,1,5,2
3,2015-01-06 00:00:00+05:30,321.85,331.45,315.60,321.64,3672197,2015,1,6,2
4,2015-01-07 00:00:00+05:30,321.10,328.70,317.39,321.95,2981544,2015,1,7,2


In [43]:
data_df['day_profit'] = data_df['close'].diff(1)
data_df.iloc[0, data_df.columns.get_loc("day_profit")] = 0
data_df.head()

,date,close,high,low,open,volume,year,month,day,week,day_profit
0,2015-01-01 00:00:00+05:30,319.55,322.50,316.25,319.00,1456204,2015,1,1,1,0.00
1,2015-01-02 00:00:00+05:30,319.35,325.80,318.05,319.35,2894058,2015,1,2,1,-0.20
2,2015-01-05 00:00:00+05:30,323.80,327.50,319.35,320.45,2099786,2015,1,5,2,4.45
3,2015-01-06 00:00:00+05:30,321.85,331.45,315.60,321.64,3672197,2015,1,6,2,-1.95
4,2015-01-07 00:00:00+05:30,321.10,328.70,317.39,321.95,2981544,2015,1,7,2,-0.75


In [80]:
class StatsCalculation:

    def __init__(self):
        self.reset()

    def reset(self):
        self.win_streak = 0
        self.lose_streak = 0

    def get_streaks(self, input_data):
        if input_data:
            self.win_streak += 1
            self.lose_streak = 0
        else:
            self.lose_streak += 1
            self.win_streak = 0
        return self.win_streak, self.lose_streak

    def maximum_info(self, input_df, column_name):
        return round(input_df[column_name].max(), 2), input_df.loc[input_df[column_name].idxmax(), 'date']

    def minimum_info(self, input_df, column_name):
        return round(input_df[column_name].min(), 2), input_df.loc[input_df[column_name].idxmin(), 'date']

    def mean_info(self, input_df, column_name):
        return round(input_df[column_name].mean(), 2)

    def median_info(self, input_df, column_name):
        return round(input_df[column_name].median(), 2)

    def max_streaks(self, input_df, column_name='win_streaks'):
        return round(input_df[column_name].max(), 2), input_df.loc[input_df[column_name].idxmax(), 'date']

    def get_stats(self, input_df, column_name='close'):
        output_dict = {}
        self.reset()
        temp = input_df["profit_flag"].apply(lambda x: self.get_streaks(x))
        input_df["win_streaks"] = temp.apply(lambda x: x[0])
        input_df["lose_streaks"] = temp.apply(lambda x: x[1])
        del temp
        output_dict['MinimumValue'], output_dict['MinimumValueDate'] = self.minimum_info(input_df, column_name)
        output_dict['MaximumValue'], output_dict['MaximumValueDate'] = self.maximum_info(input_df, column_name)
        output_dict['MeanValue'] = self.mean_info(input_df, column_name)
        output_dict['MedianValue'] = self.median_info(input_df, column_name)
        output_dict['MaxWinStreaks'], output_dict['MaxWinStreaksDate'] = self.max_streaks(input_df, "win_streaks")
        output_dict['MaxLoseStreaks'], output_dict['MaxLoseStreaksDate'] = self.max_streaks(input_df, "lose_streaks")
        return output_dict


In [86]:
statcalc = StatsCalculation()
yearly_dict = {}
data_df['year'],data_df['month'],data_df['day'],data_df['week'] =  data_df['date'].dt.year, data_df['date'].dt.month, data_df['date'].dt.day, data_df['date'].dt.isocalendar().week
for this_year in data_df['year'].unique():
    this_year_df = data_df[data_df['year'] == this_year]
    data_df['day_profit'] = data_df['close'].diff(1).fillna(0)
    this_year_df["profit_flag"] = this_year_df['day_profit'].apply(lambda x: int(x> 0))
    yearly_dict[this_year] = statcalc.get_stats(this_year_df, 'close')
pd.DataFrame(yearly_dict)

,2015,2016,2017,2018,2019,2020,2021,2022
MinimumValue,241.15,171.55,273.8,301.8,323.5,207.8,496.8,654.85
MinimumValueDate,2015-12-11 00:00:00+05:30,2016-05-19 00:00:00+05:30,2017-01-02 00:00:00+05:30,2018-10-05 00:00:00+05:30,2019-02-26 00:00:00+05:30,2020-03-23 00:00:00+05:30,2021-01-06 00:00:00+05:30,2022-02-24 00:00:00+05:30
MaximumValue,368.55,312.8,437.55,448.75,426.4,485.4,878.6,970.25
MaximumValueDate,2015-08-20 00:00:00+05:30,2016-10-25 00:00:00+05:30,2017-11-06 00:00:00+05:30,2018-01-24 00:00:00+05:30,2019-06-14 00:00:00+05:30,2020-12-30 00:00:00+05:30,2021-06-07 00:00:00+05:30,2022-09-20 00:00:00+05:30
MeanValue,314.57,238.7,360.06,376.93,380.33,349.88,709.48,772.74
MedianValue,319.14,233.6,366.58,377.25,378.1,350.15,725.82,752.85
MaxWinStreaks,7,5,6,7,5,7,6,7
MaxWinStreaksDate,2015-08-06 00:00:00+05:30,2016-07-25 00:00:00+05:30,2017-04-05 00:00:00+05:30,2018-11-26 00:00:00+05:30,2019-03-13 00:00:00+05:30,2020-08-17 00:00:00+05:30,2021-02-18 00:00:00+05:30,2022-04-06 00:00:00+05:30
MaxLoseStreaks,8,6,5,9,6,7,8,6
MaxLoseStreaksDate,2015-11-05 00:00:00+05:30,2016-01-11 00:00:00+05:30,2017-02-17 00:00:00+05:30,2018-05-24 00:00:00+05:30,2019-01-28 00:00:00+05:30,2020-03-02 00:00:00+05:30,2021-06-17 00:00:00+05:30,2022-01-24 00:00:00+05:30


# Closest Stocks

In [5]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
stock_data_folder = r"E:\PythonCodes\StockAnalysis\Data\IndianStocks_IntradayData"
file_list = [os.path.join(stock_data_folder, x) for x in os.listdir(stock_data_folder) if x.endswith('_day_data.csv')]

In [30]:
all_stock_df = pd.DataFrame()
for i in range(0, len(file_list)):
    data_df = pd.read_csv(file_list[i])
    data_df = data_df.drop(data_df.columns[0], axis=1)
    data_df['date'] = pd.to_datetime(data_df['date'])
    data_df.set_index('date', inplace=True)
    stock_variation = data_df['close'].pct_change().fillna(0)
    stock_variation.name = os.path.basename(file_list[i]).split('_')[0]
    all_stock_df = pd.concat((all_stock_df, stock_variation), axis=1)
all_stock_df.fillna(0, inplace=True)
all_stock_df.head()

,ACC,ADANIENT,ADANIGREEN,ADANIPORTS,AMBUJACEM,APOLLOHOSP,ASIANPAINT,AUROPHARMA,AXISBANK,BAJAJ-AUTO,...,TATASTEEL,TCS,TECHM,TITAN,TORNTPHARM,ULTRACEMCO,UPL,VEDL,WIPRO,YESBANK
2015-01-01 00:00:00+05:30,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-01-02 00:00:00+05:30,0.015924,0.004898,0.0,-0.000626,0.014050,0.001723,0.034573,0.004791,0.022344,-0.000163,...,0.014833,0.013309,0.004887,0.007899,0.041030,0.029897,0.031175,0.013911,0.008860,0.025435
2015-01-05 00:00:00+05:30,0.005435,0.016552,0.0,0.013935,-0.001299,0.003325,-0.000065,-0.000477,0.006319,0.003646,...,0.014616,-0.015189,-0.016677,0.008098,-0.004444,0.002042,-0.018735,-0.011246,0.001814,0.002146
2015-01-06 00:00:00+05:30,-0.034946,-0.007592,0.0,-0.006022,-0.036852,-0.029772,-0.023871,-0.039012,-0.035727,-0.007799,...,-0.048019,-0.036870,-0.010734,-0.034206,-0.045991,-0.028060,-0.048384,-0.048908,-0.023393,-0.028026
2015-01-07 00:00:00+05:30,-0.011637,-0.008858,0.0,-0.002330,-0.009228,0.007833,0.020093,0.015867,-0.000802,0.002331,...,-0.018916,-0.011812,-0.004668,0.005098,-0.005386,-0.003950,-0.018012,-0.000239,-0.007610,-0.018532


In [31]:
similarity_df = pd.DataFrame(cosine_similarity(all_stock_df.T), columns= all_stock_df.columns, index=all_stock_df.columns)
similarity_df

,ACC,ADANIENT,ADANIGREEN,ADANIPORTS,AMBUJACEM,APOLLOHOSP,ASIANPAINT,AUROPHARMA,AXISBANK,BAJAJ-AUTO,...,TATASTEEL,TCS,TECHM,TITAN,TORNTPHARM,ULTRACEMCO,UPL,VEDL,WIPRO,YESBANK
ACC,1.000000,0.303024,0.192941,0.376936,0.795596,0.283135,0.383805,0.308055,0.372170,0.377736,...,0.408042,0.219952,0.230885,0.326746,0.208425,0.657960,0.326485,0.331437,0.241182,0.136690
ADANIENT,0.303024,1.000000,0.231362,0.449356,0.316112,0.212683,0.223940,0.228647,0.284010,0.236006,...,0.334239,0.172175,0.205284,0.246649,0.171290,0.334841,0.254128,0.305640,0.198347,0.134725
ADANIGREEN,0.192941,0.231362,1.000000,0.252628,0.191118,0.115361,0.126207,0.118027,0.181436,0.088550,...,0.200827,0.079926,0.084466,0.112062,0.087955,0.197849,0.190607,0.162728,0.054165,0.126484
ADANIPORTS,0.376936,0.449356,0.252628,1.000000,0.354604,0.242976,0.311736,0.306227,0.382585,0.342973,...,0.405475,0.217086,0.279858,0.291066,0.183680,0.412648,0.329479,0.375939,0.206246,0.185171
AMBUJACEM,0.795596,0.316112,0.191118,0.354604,1.000000,0.283545,0.361460,0.304597,0.375718,0.355925,...,0.397811,0.210038,0.248139,0.329449,0.210255,0.633948,0.298365,0.335355,0.268278,0.097060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ULTRACEMCO,0.657960,0.334841,0.197849,0.412648,0.633948,0.301835,0.455751,0.313269,0.431326,0.368140,...,0.441568,0.253751,0.289476,0.337125,0.214904,1.000000,0.370110,0.366307,0.289788,0.116183
UPL,0.326485,0.254128,0.190607,0.329479,0.298365,0.214786,0.308607,0.298892,0.352929,0.304954,...,0.349794,0.255600,0.291072,0.296406,0.189226,0.370110,1.000000,0.350926,0.237812,0.100235
VEDL,0.331437,0.305640,0.162728,0.375939,0.335355,0.241208,0.200716,0.311394,0.359091,0.290966,...,0.629246,0.185430,0.221480,0.274416,0.203362,0.366307,0.350926,1.000000,0.224058,0.145526
WIPRO,0.241182,0.198347,0.054165,0.206246,0.268278,0.247667,0.214778,0.250870,0.213371,0.240483,...,0.264335,0.464338,0.481232,0.249315,0.238315,0.289788,0.237812,0.224058,1.000000,-0.065714


In [32]:
similarity_df.to_csv('./../data/similarity.csv')

In [23]:
similarity_df = pd.read_csv('./../data/similarity.csv')
similarity_df.rename(columns={'Unnamed: 0': 'StockName'}, inplace=True)
similarity_df.set_index('StockName', inplace=True)
similarity_df

,ACC,ADANIENT,ADANIGREEN,ADANIPORTS,AMBUJACEM,APOLLOHOSP,ASIANPAINT,AUROPHARMA,AXISBANK,BAJAJ-AUTO,...,TATASTEEL,TCS,TECHM,TITAN,TORNTPHARM,ULTRACEMCO,UPL,VEDL,WIPRO,YESBANK
StockName,,,,,,,,,,,,,,,,,,,,,
ACC,1.000000,0.303024,0.192941,0.376936,0.795596,0.283135,0.383805,0.308055,0.372170,0.377736,...,0.408042,0.219952,0.230885,0.326746,0.208425,0.657960,0.326485,0.331437,0.241182,0.136690
ADANIENT,0.303024,1.000000,0.231362,0.449356,0.316112,0.212683,0.223940,0.228647,0.284010,0.236006,...,0.334239,0.172175,0.205284,0.246649,0.171290,0.334841,0.254128,0.305640,0.198347,0.134725
ADANIGREEN,0.192941,0.231362,1.000000,0.252628,0.191118,0.115361,0.126207,0.118027,0.181436,0.088550,...,0.200827,0.079926,0.084466,0.112062,0.087955,0.197849,0.190607,0.162728,0.054165,0.126484
ADANIPORTS,0.376936,0.449356,0.252628,1.000000,0.354604,0.242976,0.311736,0.306227,0.382585,0.342973,...,0.405475,0.217086,0.279858,0.291066,0.183680,0.412648,0.329479,0.375939,0.206246,0.185171
AMBUJACEM,0.795596,0.316112,0.191118,0.354604,1.000000,0.283545,0.361460,0.304597,0.375718,0.355925,...,0.397811,0.210038,0.248139,0.329449,0.210255,0.633948,0.298365,0.335355,0.268278,0.097060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ULTRACEMCO,0.657960,0.334841,0.197849,0.412648,0.633948,0.301835,0.455751,0.313269,0.431326,0.368140,...,0.441568,0.253751,0.289476,0.337125,0.214904,1.000000,0.370110,0.366307,0.289788,0.116183
UPL,0.326485,0.254128,0.190607,0.329479,0.298365,0.214786,0.308607,0.298892,0.352929,0.304954,...,0.349794,0.255600,0.291072,0.296406,0.189226,0.370110,1.000000,0.350926,0.237812,0.100235
VEDL,0.331437,0.305640,0.162728,0.375939,0.335355,0.241208,0.200716,0.311394,0.359091,0.290966,...,0.629246,0.185430,0.221480,0.274416,0.203362,0.366307,0.350926,1.000000,0.224058,0.145526


In [34]:
temp = similarity_df.loc['ACC'].sort_values(ascending=False).iloc[1:6].to_frame().reset_index()
temp.columns = ['Stocks', 'Similarity Score']
temp

,Stocks,Similarity Score
0,AMBUJACEM,0.795596
1,ULTRACEMCO,0.657960
2,NIFTY 50,0.561174
3,SHREECEM,0.534265
4,GRASIM,0.527713


# Technical - MVA CrossOver Detection

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
stock_data_folder = r"E:\PythonCodes\StockAnalysis\Data\IndianStocks_IntradayData"
file_list = [os.path.join(stock_data_folder, x) for x in os.listdir(stock_data_folder) if x.endswith('_day_data.csv')]
file_path = file_list[0]

In [28]:
data_df = pd.read_csv(file_path)
data_df = data_df.drop(data_df.columns[0], axis=1)
# data_df['date'] = pd.to_datetime(data_df['date'])
# data_df.set_index('date', inplace=True)
data_df.head()

,date,close,high,low,open,volume
0,2015-01-01 00:00:00+05:30,1403.55,1411.00,1397.95,1400.15,101070
1,2015-01-02 00:00:00+05:30,1425.90,1431.00,1405.55,1406.40,153349
2,2015-01-05 00:00:00+05:30,1433.65,1439.75,1423.75,1425.00,207979
3,2015-01-06 00:00:00+05:30,1383.55,1428.95,1370.55,1425.00,368755
4,2015-01-07 00:00:00+05:30,1367.45,1384.95,1336.20,1382.00,301895


In [58]:
mva_shorter = data_df['close'].rolling(window=20).mean().fillna(0)
mva_longer = data_df['close'].rolling(window=50).mean().fillna(0)
crossover = mva_shorter - mva_longer
crossover.iloc[100:150]

100   -46.9800
101   -44.4470
102   -39.9420
103   -37.1375
104   -34.7925
105   -35.1925
106   -32.8230
107   -31.7775
108   -32.3000
109   -32.1900
110   -35.3200
111   -36.8645
112   -37.3435
113   -38.3025
114   -38.7840
115   -38.2840
116   -36.2535
117   -35.2075
118   -34.2190
119   -35.9295
120   -38.9615
121   -39.0900
122   -38.1945
123   -36.2470
124   -33.1880
125   -28.8850
126   -26.0580
127   -23.5570
128   -19.7015
129   -16.9690
130   -11.7380
131    -7.3315
132    -4.6870
133    -1.4285
134     0.8640
135     3.1235
136     2.9530
137     4.9915
138     6.8330
139     7.4975
140     8.1845
141     6.6865
142     4.5555
143     2.6635
144     0.6495
145    -2.9410
146    -6.9880
147    -9.6415
148   -12.3195
149   -13.6920
Name: close, dtype: float64

In [61]:
class MovingAverageTrader:
    
    def __init__(self):
        self.prev_sign = 0 
        self.sell_position = 0
        self.buy_position = 0
        self.net_trade = 0

    def decide(self, input_data):
        trade_move = 0
        if input_data is None:
            self.prev_sign = 0
        elif input_data > 0:
            if self.prev_sign in [0, -1]:
                # print("Buy Sign")
                trade_move = 1
                self.buy_position += 1
            self.prev_sign = 1
        elif input_data < 0:
            if self.prev_sign in [0, 1]:
                # print("Sell Sign")
                trade_move = -1
                self.sell_position += 1
            self.prev_sign = -1
        else:
            trade_move = 0
            # Prevsign will remain same if diff is zero
        self.net_trade = self.buy_position - self.sell_position
        return trade_move


movAvgTrade = MovingAverageTrader()
crossover.apply(lambda x: movAvgTrade.decide(x)).replace(0, None)

Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign


0       None
1       None
2       None
3       None
4       None
        ... 
1902    None
1903    None
1904      -1
1905    None
1906    None
Name: close, Length: 1907, dtype: object

In [30]:
np.sign(crossover).diff().ne(0)

date
2015-01-01 00:00:00+05:30     True
2015-01-02 00:00:00+05:30    False
2015-01-05 00:00:00+05:30    False
2015-01-06 00:00:00+05:30    False
2015-01-07 00:00:00+05:30    False
                             ...  
2022-10-17 00:00:00+05:30    False
2022-10-18 00:00:00+05:30    False
2022-10-19 00:00:00+05:30     True
2022-10-20 00:00:00+05:30    False
2022-10-21 00:00:00+05:30    False
Name: close, Length: 1907, dtype: bool

In [39]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = go.Figure(data=[
                        go.Candlestick(x=data_df['date'],
                                         open=data_df['open'],
                                         high=data_df['high'],
                                         low=data_df['low'],
                                         close=data_df['close']),
                        go.Scatter(x=data_df['date'],
                                   y=mva_shorter,
                                   mode="lines",
                                   name="mva_20"),
                        go.Scatter(x=data_df['date'],
                                   y=mva_longer,
                                   mode="lines",
                                   name="mva_50")

                    ]
               )

fig.

"""
fig.add_trace(
    go.Scatter(
        x=data_df['date'],
        y=mva_shorter,
        mode="lines",
        name="mva_20"
    )
)

fig.add_trace(
    go.Scatter(
        x=data_df['date'],
        y=mva_longer,
        mode="lines",
        name="mva_50"
    )
)
"""
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [62]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

movAvgTrade = MovingAverageTrader()
fig = make_subplots(rows=2, cols=1)
fig.update_layout(xaxis_rangeslider_visible=True)
fig.append_trace(go.Candlestick(x=data_df['date'],
                                open=data_df['open'],
                                high=data_df['high'],
                                low=data_df['low'],
                                close=data_df['close']
                               ), row=1, col=1)
fig.append_trace(go.Scatter(x=data_df['date'],
                            y=mva_shorter,
                            mode="lines",
                            name="mva_20"), row=1, col=1)
fig.append_trace(go.Scatter(x=data_df['date'],
                            y=mva_longer,
                            mode="lines",
                            name="mva_50"), row=1, col=1)
fig.append_trace(go.Scatter(x=data_df['date'],
                            y=crossover.apply(lambda x: movAvgTrade.decide(x)).replace(0, None),
                            mode="markers",
                            name="Buy or Sell"), row=2, col=1)
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
Buy Sign
Sell Sign
